In [1]:
from preprocessing.video import Video
from preprocessing.dto.sequence_dto import SequenceDto
from preprocessing.inference_service.external_model.yolov7_pose_tracker.utils.plots import plot_one_box_kpt, \
    plot_skeleton_kpts
from pathlib import Path
import cv2
import numpy as np
from scipy.signal import savgol_filter
from functools import partial

v = Video(Path('/home/rafa/SVW/Videos/baseball/159___46599ed4c8a743ceb9385afd5c0c1c2e.mp4'))
sequences = v.build_sequences()
sequences.head(100)

,video,frame_id,bbox,max_wh,skeleton,...,visual_features,score,person_id,frame_copied_from,start_frame
0,/home/rafa/SVW/Videos/baseball/159___46599ed4c...,2,"[437.0, 381.0, 696.0, 544.0]","[947, 684]","[1007.257080078125, 471.0394287109375, 0.96507...",...,"[21.471346, 10.253132, -0.0, -0.0, 12.765939, ...",0.80684674,1,2,2
1,/home/rafa/SVW/Videos/baseball/159___46599ed4c...,3,"[439.0, 393.0, 690.0, 531.0]","[947, 684]","[988.9431762695312, 473.6553955078125, 0.97315...",...,"[30.47338, 6.104456, -0.0, 3.1419911, 9.406569...",0.83995163,1,3,2
2,/home/rafa/SVW/Videos/baseball/159___46599ed4c...,4,"[438.0, 390.0, 689.0, 535.0]","[947, 684]","[962.67041015625, 474.9436340332031, 0.9498740...",...,"[26.214083, 10.831504, 3.1966224, 4.001185, 14...",0.79532224,1,4,2
3,/home/rafa/SVW/Videos/baseball/159___46599ed4c...,5,"[431.0, 399.0, 692.0, 527.0]","[947, 684]","[924.8689575195312, 474.8865051269531, 0.89276...",...,"[27.980362, 8.410365, 5.5522265, 2.9207, 12.39...",0.87010795,1,5,2
4,/home/rafa/SVW/Videos/baseball/159___46599ed4c...,6,"[397.0, 358.0, 722.0, 574.0]","[947, 684]","[899.876220703125, 474.7781677246094, 0.854667...",...,"[24.578773, 8.31082, 3.503761, 1.4255794, 9.57...",0.7400967,1,6,2
...,...,...,...,...,...,...,...,...,...,...,...
31,/home/rafa/SVW/Videos/baseball/159___46599ed4c...,97,"[465.0, 423.0, 871.0, 495.0]","[871, 591]","[989.4848022460938, 476.845458984375, 0.577758...",...,"[0.26552388, 10.329921, 0.62716675, 9.257057, ...",0.86383915,1,97,66
0,/home/rafa/SVW/Videos/baseball/159___46599ed4c...,66,"[561.0, 279.0, 219.0, 170.0]","[236, 219]","[635.3939208984375, 321.7694091796875, 0.89953...",...,"[12.446587, 8.3495035, 1.3306116, -0.0, 9.7622...",0.35747346,2,67,66
1,/home/rafa/SVW/Videos/baseball/159___46599ed4c...,67,"[561.0, 279.0, 219.0, 170.0]","[236, 219]","[636.154541015625, 321.9903869628906, 0.884697...",...,"[12.446587, 8.3495035, 1.3306116, -0.0, 9.7622...",0.35747346,2,67,66
2,/home/rafa/SVW/Videos/baseball/159___46599ed4c...,68,"[563.0, 280.0, 214.0, 169.0]","[236, 219]","[637.0172119140625, 322.1476745605469, 0.86541...",...,"[16.27729, 8.647038, 0.48047125, -0.0, 10.7310...",0.27248994,2,68,66


In [12]:
sequences[SequenceDto.SKELETON_ANGLES].apply(lambda val: val.shape)

0     (15,)
1     (15,)
2     (15,)
3     (15,)
4     (15,)
      ...  
27    (15,)
28    (15,)
29    (15,)
30    (15,)
31    (15,)
Name: skeleton_angles, Length: 128, dtype: object

In [13]:
cap = cv2.VideoCapture(v.video_reshaped_path.as_posix())
video = [np.zeros((1080, 1920, 3))]
while (cap.isOpened()):
    ret, img = cap.read()
    if not ret:
        break
    video.append(img)
cap.release()



In [14]:
# from preprocessing.dto.detection_dto import DetectionDto
#
# for frame_id in v.detections[DetectionDto.FRAME_ID].unique():
#     frame = video[frame_id]
#     skeletons = v.detections[v.detections[DetectionDto.FRAME_ID] == frame_id][DetectionDto.SKELETON].tolist()
#     detections = v.detections[v.detections[DetectionDto.FRAME_ID] == frame_id][DetectionDto.BBOX].tolist()
#     for skeleton, detection in zip(skeletons, detections):
#         frame_copy = frame.copy()
#         plot_one_box_kpt(x=np.array([detection[0], detection[1], detection[0]+detection[2],detection[1]+detection[3]]), im=frame_copy, kpts=skeleton, kpt_label=True,steps=3)
#         cv2.imshow('frame', frame_copy)
#         cv2.waitKey(0)
# cv2.destroyAllWindows()

In [15]:

persons = sequences[SequenceDto.PERSON_ID].unique()

for person in persons:
    person_video_chunks = sequences[sequences[SequenceDto.PERSON_ID] == person][SequenceDto.START_FRAME].unique()
    for person_video_chunk in person_video_chunks:
        print(person)
        person_sequences_df = sequences[
            (sequences[SequenceDto.PERSON_ID] == person) & (sequences[SequenceDto.START_FRAME] == person_video_chunk)]

        skeletons = person_sequences_df[SequenceDto.SKELETON_NORMALIZED]

        for skeleton in skeletons:
            skeleton = np.array(skeleton)
            # int_bbox = person_seq.bbox.astype(int)
            # frame = video[person_seq.frame_copied_from].copy()
            # plot_skeleton_kpts(im=frame, kpts=person_seq.skeleton, steps=3)
            # roi = frame[int_bbox[1]:int_bbox[1] + person_seq.max_wh[1],
            #         int_bbox[0]:int_bbox[0] + person_seq.max_wh[0]]
            # cv2.imshow(f'frame', roi)
            frame = np.zeros((100, 100, 3))
            # skeleton = person_seq.skeleton_normalized
            skeleton[0::3] *= 100
            skeleton[1::3] *= 100
            print(skeleton)
            plot_skeleton_kpts(im=frame, kpts=skeleton, steps=3)
            cv2.imshow('frame', frame)
            cv2.waitKey(0)
cv2.destroyAllWindows()

1
[     60.443      13.489     0.96507      60.547      11.137     0.78073      58.477      11.787     0.96655      59.753      11.352    0.033073      52.972      12.151     0.68755      60.012      20.902     0.91701      45.613      18.404     0.93552      67.722      23.025     0.85967       35.68      18.175
     0.83923      53.627      22.597     0.86197        39.3      18.871     0.80253      47.275      41.253     0.96481      36.197      39.837     0.96646      57.743      33.005     0.96239      22.945      53.494      0.9641      58.902      67.761     0.90835      8.4917      68.472     0.90398]


QObject::moveToThread: Current thread (0x8b1cab0) is not the object's thread (0x9a473a0).
Cannot move to target thread (0x8b1cab0)

QObject::moveToThread: Current thread (0x8b1cab0) is not the object's thread (0x9a473a0).
Cannot move to target thread (0x8b1cab0)

QObject::moveToThread: Current thread (0x8b1cab0) is not the object's thread (0x9a473a0).
Cannot move to target thread (0x8b1cab0)

QObject::moveToThread: Current thread (0x8b1cab0) is not the object's thread (0x9a473a0).
Cannot move to target thread (0x8b1cab0)

QObject::moveToThread: Current thread (0x8b1cab0) is not the object's thread (0x9a473a0).
Cannot move to target thread (0x8b1cab0)

QObject::moveToThread: Current thread (0x8b1cab0) is not the object's thread (0x9a473a0).
Cannot move to target thread (0x8b1cab0)

QObject::moveToThread: Current thread (0x8b1cab0) is not the object's thread (0x9a473a0).
Cannot move to target thread (0x8b1cab0)

QObject::moveToThread: Current thread (0x8b1cab0) is not the object's thread

[     57.674      11.512     0.97315      57.732      9.4073     0.63427      55.797      9.8403     0.96693      57.072      9.8041    0.021914      50.514      10.592     0.74511      57.501      19.214     0.93196      43.555      16.626     0.94965      68.597      22.056     0.88072      34.184      16.072
     0.89782      53.785      21.509     0.89512      40.357      17.233     0.87857      45.703      38.125     0.96897      34.789      36.799     0.97144      55.053      33.391     0.96357      21.828      51.423     0.96652      57.583      65.742      0.9072      7.8993      66.452     0.90373]
[      55.24      11.303     0.94987      55.288      9.3667     0.50911      53.447      9.6714     0.93704      54.717      9.7987    0.017492       48.37      10.523     0.75737      54.942      18.777     0.93551      41.266      16.398     0.95143      65.324      21.593     0.88308      31.556      15.838
     0.90253      50.789      21.111     0.89397       37.37      16.632